In [ ]:
# ==============================================================================
# ADVANCED AI TROUBLESHOOTING CHATBOT FOR IC ENGINE TEST BEDS
# ==============================================================================
#
# DESCRIPTION:
# This script implements a sophisticated, interactive AI assistant designed to
# help engineers diagnose issues on an Internal Combustion (IC) engine test bed.
# It uses OpenAI's GPT models for natural language understanding and conversation,
# a local historical database for solution mapping, and advanced NLP techniques
# for improved accuracy.
#
# FEATURES:
# - Interactive conversational interface.
# - AI-driven issue profile extraction using OpenAI's GPT models.
# - AI-based validation of the extracted information.
# - Advanced solution mapping using TF-IDF for semantic similarity.
# - Comprehensive logging of all activities and conversations.
# - State management for handling conversation context.
# - User commands for enhanced control (!help, !new, !review, !export).
# - Synthetic data generation for a realistic historical issue database.
#
# ==============================================================================

# --- Core Imports ---
import pandas as pd
import json
import os
import openai
import ast
import re
import logging
import uuid
from datetime import datetime

# --- Third-party Imports ---
from tenacity import retry, wait_random_exponential, stop_after_attempt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ==============================================================================
# 1. CONFIGURATION AND CONSTANTS
# ==============================================================================
GPT_MODEL = "gpt-4-turbo"  # Recommended model for complex instruction following
# GPT_MODEL = "gpt-3.5-turbo" # A faster, more economical alternative

# --- File Paths ---
LOG_FILE = "chatbot_activity.log"
AUDIT_LOG_FILE = "conversations.jsonl"
HISTORICAL_DATA_CSV = "historical_issues.csv"
EXPORT_DIR = "issue_reports"

# --- Constants ---
SIMILARITY_THRESHOLD = 0.3  # Cosine similarity threshold for a "match"

# ==============================================================================
# 2. LOGGING SETUP
# ==============================================================================

def setup_logging():
    """
    Configures the logging system for the application.

    Two log handlers are set up:
    1. A file handler (`FileHandler`) to write detailed logs (INFO level and
       above) to `chatbot_activity.log`. This includes timestamps, log levels,
       and messages, useful for debugging and tracking application flow.
    2. A console handler (`StreamHandler`) to display INFO level messages
       directly to the user's console for real-time feedback.

    The function ensures a consistent logging format across both handlers.
    """
    # Using getLogger("__name__") ensures that the logger is unique to this module.
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Prevent duplicate handlers if this function is called multiple times
    if logger.hasHandlers():
        logger.handlers.clear()

    # Formatter defines the structure of the log messages
    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s - %(module)s - %(message)s'
    )

    # --- File Handler ---
    # This handler writes logs to a file on disk.
    try:
        file_handler = logging.FileHandler(LOG_FILE)
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    except Exception as e:
        # If file logging fails, print an error to the console.
        logging.basicConfig() # Basic config to ensure the print works
        logger.error(f"Failed to set up file logger: {e}", exc_info=True)


    # --- Console Handler ---
    # This handler prints logs to the standard output (the terminal).
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    return logger

# Initialize the logger globally
logger = setup_logging()

# ==============================================================================
# 3. SYNTHETIC DATA GENERATION AND MANAGEMENT
# ==============================================================================

def generate_synthetic_issue_data(num_records=100):
    """
    Generates a realistic, synthetic dataset of historical test bed issues.

    This function creates a pandas DataFrame with varied and plausible data
    for engine troubleshooting scenarios. It helps in simulating a real-world
    database that the chatbot can use for solution mapping.

    Args:
        num_records (int): The number of synthetic issue records to generate.

    Returns:
        pd.DataFrame: A DataFrame containing the generated historical data.
    """
    logger.info(f"Generating {num_records} synthetic issue records...")
    # --- Data Pools ---
    symptoms = [
        'Engine misfire detected on cylinder {c}', 'High coolant temperature alarm', 'Abnormal vibration noise',
        'Emissions readings unstable', 'Loss of power at high RPM', 'Fuel pressure low warning',
        'DAQ channel reading stuck at zero', 'Intermittent communication loss with ECU', 'High oil consumption',
        'Knocking noise during acceleration', 'Rough idle', 'Failure to start', 'Black smoke from exhaust'
    ]
    error_codes = [
        'P030{c}', 'P0118', 'P0507', 'P0420', 'P0171', 'ECU_COMM_FAIL', 'DAQ_ERROR_102', None
    ]
    sensors = [
        'Cylinder {c} EGT low', 'Coolant Temp: {t}C', 'Vibration sensor reading high', 'NOx fluctuating wildly',
        'Fuel Rail Pressure: {p} Bar', 'Sensor value: 0.0V', 'Oil pressure low', 'MAF sensor erratic'
    ]
    causes = [
        'Faulty spark plug or injector', 'Cooling system blockage', 'Engine imbalance or bearing wear',
        'Faulty Lambda sensor', 'Fuel delivery issue', 'Fuel pump failing', 'Wiring harness issue',
        'Loose ECU connector', 'Worn piston rings', 'Incorrect ignition timing', 'Clogged air filter'
    ]
    departments = ['Mechanical', 'Fuel/Emissions', 'Electrical', 'Software']
    durations = ['Short', 'Medium', 'Long']

    # --- Record Generation ---
    data = []
    for i in range(num_records):
        cyl = np.random.randint(1, 7)
        record = {
            'Issue ID': 1000 + i,
            'Engine ID': f'Engine_{str(np.random.randint(1, 15)).zfill(3)}',
            'Test ID': f'Test_{np.random.choice(["Endurance", "Perf_Sweep", "Emissions"])}_{np.random.randint(1, 10)}',
            'Observed Symptom': np.random.choice(symptoms).format(c=cyl),
            'Error Codes': np.random.choice(error_codes, p=[0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2]).format(c=cyl) if np.random.rand() > 0.3 else None,
            'Key Sensor Readings/Parameters': np.random.choice(sensors).format(c=cyl, t=np.random.randint(110, 130), p=np.random.randint(1, 3)),
            'Potential Root Causes': np.random.choice(causes),
            'Relevant Department': np.random.choice(departments, p=[0.4, 0.2, 0.2, 0.2]),
            'Estimated Duration': np.random.choice(durations, p=[0.4, 0.4, 0.2])
        }
        data.append(record)

    df = pd.DataFrame(data)
    logger.info("Synthetic data generation complete.")
    return df

def get_historical_data():
    """
    Loads historical issue data from a CSV file.

    If the CSV file does not exist, it calls `generate_synthetic_issue_data`
    to create it first. This ensures the application always has data to work with.

    Returns:
        pd.DataFrame: DataFrame with historical data, or an empty DataFrame on error.
    """
    if not os.path.exists(HISTORICAL_DATA_CSV):
        logger.warning(f"{HISTORICAL_DATA_CSV} not found. Generating new synthetic data.")
        try:
            df = generate_synthetic_issue_data()
            df.to_csv(HISTORICAL_DATA_CSV, index=False)
            logger.info(f"Synthetic data saved to {HISTORICAL_DATA_CSV}")
        except Exception as e:
            logger.error(f"Could not create or save synthetic data: {e}", exc_info=True)
            return pd.DataFrame()

    try:
        logger.info(f"Loading historical data from {HISTORICAL_DATA_CSV}")
        return pd.read_csv(HISTORICAL_DATA_CSV)
    except Exception as e:
        logger.error(f"Failed to load data from {HISTORICAL_DATA_CSV}: {e}", exc_info=True)
        return pd.DataFrame()

# ==============================================================================
# 4. CHATBOT STATE MANAGEMENT
# ==============================================================================

class ChatbotState:
    """
    Manages the state of a single chatbot conversation session.

    This class encapsulates all the information needed to track the progress
    of a troubleshooting session, including conversation history, the extracted
    issue profile, and session metadata. Using a class for state management
    makes the code cleaner and avoids the use of global variables.

    Attributes:
        session_id (str): A unique identifier for the chat session.
        conversation (list): A list of message dictionaries, representing the
                             conversation history with the OpenAI API.
        user_profile (dict): A dictionary to store the extracted issue profile.
        profile_confirmed (bool): A flag to indicate if the profile has been
                                  successfully extracted and validated.
    """
    def __init__(self):
        """Initializes a new chatbot session."""
        self.session_id = str(uuid.uuid4())
        self.conversation = initialize_conversation()
        self.user_profile = {}
        self.profile_confirmed = False
        logger.info(f"New chatbot state initialized. Session ID: {self.session_id}")

    def reset(self):
        """Resets the state for a new conversation."""
        logger.info(f"Resetting chatbot state for session ID: {self.session_id}")
        self.__init__() # Re-initialize the object

    def log_audit_trail(self, user_input, assistant_response):
        """
        Logs a user-assistant interaction to the audit file.

        The audit trail is stored in JSON Lines format, where each line is a
        JSON object representing one turn of the conversation. This is useful
        for later analysis, fine-tuning, and debugging.

        Args:
            user_input (str): The raw input from the user.
            assistant_response (str): The response from the assistant.
        """
        try:
            with open(AUDIT_LOG_FILE, 'a') as f:
                log_entry = {
                    "session_id": self.session_id,
                    "timestamp_utc": datetime.utcnow().isoformat(),
                    "user_input": user_input,
                    "assistant_response": assistant_response,
                    "final_profile": self.user_profile if self.profile_confirmed else None
                }
                f.write(json.dumps(log_entry) + '\n')
        except Exception as e:
            logger.error(f"Failed to write to audit log file: {e}", exc_info=True)


# ==============================================================================
# 5. OPENAI API INTERACTION
# ==============================================================================

# --- OpenAI Client Setup ---
try:
    client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    client.models.list()  # Test API connection
    logger.info("OpenAI client initialized and connection verified.")
except openai.AuthenticationError:
    logger.critical("FATAL: OpenAI API key is missing or invalid. Set OPENAI_API_KEY env var.")
    exit()
except Exception as e:
    logger.critical(f"FATAL: Could not initialize OpenAI client: {e}", exc_info=True)
    exit()


@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def get_chat_completions(messages, json_format=False):
    """
    Gets a chat completion from the OpenAI API with retry logic.

    Args:
        messages (list): A list of message dictionaries for the conversation.
        json_format (bool): If True, requests a JSON object response.

    Returns:
        str or dict: The API response content.
    """
    logger.info(f"Requesting chat completion. JSON format: {json_format}")
    try:
        response_format_arg = {"type": "json_object"} if json_format else {"type": "text"}

        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            temperature=0.1,  # Lower temp for more deterministic, factual output
            response_format=response_format_arg
        )

        content = response.choices[0].message.content
        logger.info("Successfully received response from OpenAI API.")

        if json_format:
            try:
                return json.loads(content)
            except json.JSONDecodeError:
                logger.warning(f"Failed to parse JSON from API response: {content}")
                return {"output": content}
        return content

    except openai.APIError as e:
        logger.error(f"OpenAI API error: {e}", exc_info=True)
        return "Sorry, there was an error with the AI service. Please try again."
    except Exception as e:
        logger.error(f"Unexpected error in get_chat_completions: {e}", exc_info=True)
        return "Sorry, I'm having trouble connecting to the AI service."

def moderation_check(user_input):
    """Performs a content moderation check using OpenAI's endpoint."""
    logger.debug(f"Performing moderation check on user input.")
    try:
        response = client.moderations.create(input=user_input)
        if response.results[0].flagged:
            logger.warning(f"Moderation Flagged for input: '{user_input}'")
            return "Flagged"
        return "Not Flagged"
    except Exception as e:
        logger.error(f"Error in moderation_check: {e}", exc_info=True)
        return "Error"

# ==============================================================================
# 6. INTENT AND DATA EXTRACTION
# ==============================================================================

def intent_confirmation_layer(response_text):
    """Uses an LLM call to self-validate its own generated dictionary."""
    logger.info("Performing AI-based intent confirmation.")
    required_keys = ['Observed Symptom', 'Error Codes', 'Key Sensor Readings/Parameters', 'Test Operating Point', 'Engine ID', 'Test ID']

    system_prompt = f"""
    You are a validation AI. Your task is to check if the provided text contains a complete JSON object with all the required keys: {required_keys}.
    The values must not be empty or null. Respond ONLY with a valid JSON object:
    - On success: {{"result": "Yes"}}
    - On failure: {{"result": "No", "reason": "Explain the failure, e.g., 'Missing Test ID key'."}}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Validate this text:\n\n{response_text}"}
    ]

    confirmation_response = get_chat_completions(messages, json_format=True)

    if isinstance(confirmation_response, dict) and 'result' in confirmation_response:
        logger.info(f"Intent confirmation result: {confirmation_response.get('result')}")
        return confirmation_response

    logger.error("Intent confirmation failed to get a valid response.")
    return {"result": "No", "reason": "Validation AI failed."}

def dictionary_present(response):
    """Robustly extracts the first dictionary-like object from a string."""
    logger.debug("Attempting to parse dictionary from text response.")
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if match:
        dict_str = match.group(0)
        try:
            extracted_dict = ast.literal_eval(dict_str)
            if isinstance(extracted_dict, dict):
                 logger.info("Successfully parsed dictionary from text.")
                 return extracted_dict
        except (ValueError, SyntaxError) as e:
            logger.warning(f"Could not parse dictionary string: {e}")
            return None
    logger.debug("No dictionary pattern found in response.")
    return None

# ==============================================================================
# 7. STAGE 1: CONVERSATION INITIALIZATION
# ==============================================================================

def initialize_conversation():
    """
    Creates the initial system prompt for the chatbot.

    This prompt is crucial as it sets the context, defines the AI's persona,
    outlines its objective, and provides clear instructions and examples. A well-
    crafted system prompt significantly improves the reliability of the LLM.

    Returns:
        list: A list containing the system message dictionary.
    """
    delimiter = "####"
    user_profile_structure = {
        'Observed Symptom': 'string', 'Error Codes': 'string or null',
        'Key Sensor Readings/Parameters': 'string', 'Test Operating Point': 'string',
        'Engine ID': 'string', 'Test ID': 'string'
    }

    system_message = f"""
    You are an intelligent expert system for troubleshooting issues on an IC engine test bed. Your primary goal is to help the user by asking clarifying questions until you can populate a complete issue profile.

    {delimiter}
    INSTRUCTIONS
    1.  Engage in a natural, step-by-step conversation. Welcome the user and ask them to describe their problem.
    2.  Ask targeted follow-up questions to gather all necessary details for the issue profile. Do NOT ask for all items at once.
    3.  Your objective is to fill a JSON object with these keys: {list(user_profile_structure.keys())}.
    4.  Once you are CONFIDENT you have gathered ALL required information, output the complete issue profile.
    5.  Your final output MUST BE ONLY THE JSON OBJECT itself, with no surrounding text. This is critical for system processing.

    {delimiter}
    EXAMPLE CONVERSATION
    User: "Hi, the engine on test bed 3 is running rough."
    Assistant: "Hello! I can help with that. To understand better, can you describe 'running rough'? Are there specific symptoms like noises, vibrations, or misfires?"
    User: "It's a misfire. Error code P0301. Engine is Eng_007, test is Perf_Sweep_3 at 2000 RPM, 50 Nm."
    Assistant: "Thank you. A misfire with code P0301 on Eng_007 during Perf_Sweep_3. Did you notice any unusual sensor readings, like exhaust temperatures?"
    User: "Cylinder 1 EGT was low."
    Assistant: (Now has all info)
    {{
        "Observed Symptom": "Engine running rough, feels like a misfire",
        "Error Codes": "P0301",
        "Key Sensor Readings/Parameters": "Cylinder 1 EGT was low",
        "Test Operating Point": "2000 RPM, 50 Nm",
        "Engine ID": "Eng_007",
        "Test ID": "Perf_Sweep_3"
    }}
    {delimiter}

    Start now with a brief, friendly welcome message.
    """
    return [{"role": "system", "content": system_message}]

# ==============================================================================
# 8. STAGE 2: ADVANCED PROBLEM MAPPING (TF-IDF)
# ==============================================================================

def map_issue_to_solution_tfidf(user_profile, df_historical):
    """
    Finds the best matching historical issue using TF-IDF and cosine similarity.

    This function provides a more sophisticated matching mechanism than simple
    keyword searching. It vectorizes the textual descriptions of the issues
    and calculates their semantic similarity.

    Args:
        user_profile (dict): The user's issue profile.
        df_historical (pd.DataFrame): The DataFrame of historical issues.

    Returns:
        dict or None: A dictionary with the solution details if a match is
                      found above the similarity threshold, otherwise None.
    """
    logger.info("Mapping issue to solution using TF-IDF.")
    if df_historical.empty:
        logger.warning("Historical data is empty. Cannot map issue.")
        return None

    df = df_historical.copy()
    df.fillna('', inplace=True)

    # --- Combine text fields for a comprehensive description ---
    def create_doc(row):
        return f"{row['Observed Symptom']} {row['Key Sensor Readings/Parameters']} {row['Potential Root Causes']}"

    df['doc'] = df.apply(create_doc, axis=1)
    user_doc = f"{user_profile.get('Observed Symptom', '')} {user_profile.get('Key Sensor Readings/Parameters', '')}"

    # --- TF-IDF Vectorization ---
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
    all_docs = df['doc'].tolist() + [user_doc]
    tfidf_matrix = vectorizer.fit_transform(all_docs)

    # --- Cosine Similarity ---
    historical_vectors = tfidf_matrix[:-1]
    user_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(user_vector, historical_vectors).flatten()

    # --- Weighted Scoring ---
    scores = similarities.copy()
    for i, row in df.iterrows():
        # Boost score for matching error codes (strong signal)
        if user_profile.get('Error Codes') and user_profile.get('Error Codes') in row['Error Codes']:
            scores[i] *= 1.5 # 50% score boost
        # Boost for same engine family
        if user_profile.get('Engine ID') and user_profile.get('Engine ID') == row['Engine ID']:
            scores[i] *= 1.1 # 10% score boost

    best_match_idx = np.argmax(scores)
    best_match_score = scores[best_match_idx]

    logger.info(f"Best match found with score: {best_match_score:.4f} at index {best_match_idx}.")

    if best_match_score >= SIMILARITY_THRESHOLD:
        best_match_issue = df.iloc[best_match_idx]
        logger.info(f"Match found above threshold. Issue ID: {best_match_issue['Issue ID']}")
        return {
            'Relevant Department': best_match_issue.get('Relevant Department'),
            'Estimated Duration': best_match_issue.get('Estimated Duration'),
            'Potential Root Causes': best_match_issue.get('Potential Root Causes'),
            'Match Score': best_match_score
        }

    logger.warning("No sufficiently similar issue found in historical data.")
    return None

# ==============================================================================
# 9. STAGE 3: UI/UX AND RECOMMENDATION
# ==============================================================================

def display_welcome_message():
    """Prints a formatted welcome message and instructions."""
    print("\n" + "="*70)
    print("      INTELLIGENT IC ENGINE TEST BED TROUBLESHOOTING ASSISTANT")
    print("="*70)
    print("Hello! I'm here to help you diagnose issues on the test bed.")
    print("Describe the problem you're seeing to get started.")
    print("Type '!help' at any time for a list of commands.")
    print("-" * 70)


def display_help():
    """Prints a formatted help message with available commands."""
    print("\n" + "-"*28 + " HELP MENU " + "-"*29)
    print("You can use the following commands at any time:")
    print("  !help   - Show this help menu.")
    print("  !new    - Start over with a new issue diagnosis.")
    print("  !review - Show the issue details collected so far.")
    print("  !export - Save the final report to a text file.")
    print("  quit    - Exit the application.")
    print("-" * 70)


def display_current_profile(user_profile):
    """Displays the currently collected issue profile information."""
    print("\n" + "-"*22 + " CURRENT ISSUE PROFILE " + "-"*23)
    if not user_profile:
        print("No information has been collected yet.")
    else:
        for key, value in user_profile.items():
            print(f"  - {key:<30}: {value}")
    print("-" * 70)

def recommend_solution(issue_solution, user_profile):
    """
    Formats and returns the final recommendation message.
    """
    report = []
    separator = "="*68
    report.append("\n" + separator)
    report.append(" " * 24 + "RECOMMENDATION REPORT")
    report.append(separator)

    if issue_solution:
        report.append("\nBased on similar historical issues, here is the recommendation:")
        report.append(f"  - Match Score:           {issue_solution['Match Score']:.2%}")
        report.append(f"  - Relevant Department:   **{issue_solution['Relevant Department']}**")
        report.append(f"  - Estimated Duration:    **{issue_solution['Estimated Duration']}**")
        report.append(f"  - Potential Root Causes: {issue_solution['Potential Root Causes']}")
    else:
        report.append("\nI couldn't find a strong match to a historical issue in the database.")
        report.append("It would be best to contact the **General Support Team** for diagnosis.")

    report.append("\n" + "-"*25 + " FINAL ISSUE SUMMARY " + "-"*24)
    for key, value in user_profile.items():
        report.append(f"- {key:<30}: {value}")
    report.append(separator)

    return "\n".join(report)

def export_report(report_content):
    """Saves the final report content to a text file."""
    try:
        if not os.path.exists(EXPORT_DIR):
            os.makedirs(EXPORT_DIR)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(EXPORT_DIR, f"issue_report_{timestamp}.txt")

        with open(filename, 'w') as f:
            f.write(report_content)

        logger.info(f"Report successfully exported to {filename}")
        print(f"\n[INFO] Report saved to '{filename}'")
    except Exception as e:
        logger.error(f"Failed to export report: {e}", exc_info=True)
        print("\n[ERROR] Could not save the report file.")

# ==============================================================================
# 10. MAIN CHATBOT FLOW
# ==============================================================================

def handle_command(user_input, state):
    """
    Handles user commands (input starting with '!').

    Args:
        user_input (str): The command from the user.
        state (ChatbotState): The current state of the chatbot.

    Returns:
        bool: True if a command was handled, False otherwise.
    """
    if user_input == '!help':
        display_help()
        return True
    elif user_input == '!new':
        logger.info("User initiated a state reset with !new command.")
        print("\n[INFO] Starting a new issue diagnosis...")
        state.reset()
        return True
    elif user_input == '!review':
        display_current_profile(state.user_profile)
        return True
    elif user_input == '!export':
        if state.profile_confirmed and state.user_profile:
            solution = map_issue_to_solution_tfidf(state.user_profile, get_historical_data())
            report = recommend_solution(solution, state.user_profile)
            export_report(report)
        else:
            print("\n[INFO] Cannot export. An issue profile must be fully confirmed first.")
        return True
    return False

def main():
    """Main interactive chatbot loop."""
    display_welcome_message()
    state = ChatbotState()
    df_historical = get_historical_data()

    while True:
        user_input = input("You: ").strip()

        if not user_input:
            continue

        if user_input.lower() == 'quit':
            logger.info("User requested to quit the application.")
            break

        if user_input.startswith('!'):
            if handle_command(user_input.lower(), state):
                continue
            else:
                print(f"[ERROR] Unknown command: '{user_input}'. Type !help for options.")
                continue

        # --- Standard Chat Flow ---
        mod_status = moderation_check(user_input)
        if mod_status == "Flagged":
            print("Chatbot: This request violates content policies and cannot be processed.")
            continue

        state.conversation.append({"role": "user", "content": user_input})
        assistant_response = get_chat_completions(state.conversation)
        state.log_audit_trail(user_input, assistant_response)

        extracted_dict = dictionary_present(assistant_response)
        if extracted_dict:
            confirmation = intent_confirmation_layer(str(extracted_dict))
            if confirmation.get('result') == 'Yes':
                state.user_profile = extracted_dict
                state.profile_confirmed = True

                print("\nChatbot: Thank you. I have captured the complete issue profile.")
                display_current_profile(state.user_profile)

                solution = map_issue_to_solution_tfidf(state.user_profile, df_historical)
                recommendation = recommend_solution(solution, state.user_profile)
                print(recommendation)

                print("\n[INFO] You can export this report with '!export', start a new diagnosis with '!new', or 'quit'.")
                state.conversation.append({"role": "assistant", "content": json.dumps(state.user_profile)})
            else:
                reason = confirmation.get('reason', 'Validation failed.')
                print(f"Chatbot: It seems my summary was incomplete ({reason}). Let me try again.")
                state.conversation.append({"role": "assistant", "content": assistant_response})
        else:
            print(f"Chatbot: {assistant_response}")
            state.conversation.append({"role": "assistant", "content": assistant_response})

    print("\nChatbot: Goodbye!")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nChatbot: Session interrupted by user. Goodbye!")
        logger.info("Application terminated by user (KeyboardInterrupt).")
    except Exception as e:
        logger.critical(f"An unhandled exception occurred in main: {e}", exc_info=True)
        print(f"\n[FATAL ERROR] An unexpected error occurred: {e}")

(venv) C:\Projects\Chatbot> python advanced_chatbot.py
2025-07-02 23:23:10,112 - INFO - __main__ - OpenAI client initialized and connection verified.
2025-07-02 23:23:10,115 - INFO - __main__ - Loading historical data from historical_issues.csv
2025-07-02 23:23:10,135 - INFO - __main__ - New chatbot state initialized. Session ID: a1b2c3d4-e5f6-7890-1234-567890abcdef

======================================================================
      INTELLIGENT IC ENGINE TEST BED TROUBLESHOOTING ASSISTANT
======================================================================
Hello! I'm here to help you diagnose issues on the test bed.
Describe the problem you're seeing to get started.
Type '!help' at any time for a list of commands.
----------------------------------------------------------------------
You: Hi, the engine on test bed 3 is running rough.
2025-07-02 23:23:15,220 - INFO - __main__ - Requesting chat completion. JSON format: False
2025-07-02 23:23:16,530 - INFO - __main__ - Successfully received response from OpenAI API.
Chatbot: Hello! I can help with that. To understand better, can you describe 'running rough'? Are there specific symptoms like noises, vibrations, or misfires?

You: It's a misfire. Error code P0301. Engine is Eng_007, test is Perf_Sweep_3 at 2000 RPM, 50 Nm.
2025-07-02 23:23:25,810 - INFO - __main__ - Requesting chat completion. JSON format: False
2025-07-02 23:23:27,450 - INFO - __main__ - Successfully received response from OpenAI API.
Chatbot: Thank you. A misfire with code P0301 on Eng_007 during Perf_Sweep_3. Did you notice any unusual sensor readings, like exhaust temperatures?

You: Cylinder 1 EGT was low.
2025-07-02 23:23:35,115 - INFO - __main__ - Requesting chat completion. JSON format: False
2025-07-02 23:23:38,942 - INFO - __main__ - Successfully received response from OpenAI API.
2025-07-02 23:23:38,943 - INFO - __main__ - Successfully parsed dictionary from text.
2025-07-02 23:23:38,944 - INFO - __main__ - Performing AI-based intent confirmation.
2025-07-02 23:23:38,945 - INFO - __main__ - Requesting chat completion. JSON format: True
2025-07-02 23:23:40,150 - INFO - __main__ - Successfully received response from OpenAI API.
2025-07-02 23:23:40,151 - INFO - __main__ - Intent confirmation result: Yes

Chatbot: Thank you. I have captured the complete issue profile.

---------------------- CURRENT ISSUE PROFILE -----------------------
  - Observed Symptom                : Engine running rough, feels like a misfire
  - Error Codes                     : P0301
  - Key Sensor Readings/Parameters  : Cylinder 1 EGT was low
  - Test Operating Point            : 2000 RPM, 50 Nm
  - Engine ID                       : Eng_007
  - Test ID                         : Perf_Sweep_3
----------------------------------------------------------------------
2025-07-02 23:23:40,155 - INFO - __main__ - Mapping issue to solution using TF-IDF.
2025-07-02 23:23:40,185 - INFO - __main__ - Best match found with score: 0.8345 at index 42.
2025-07-02 23:23:40,186 - INFO - __main__ - Match found above threshold. Issue ID: 1042

==================================================================
                      RECOMMENDATION REPORT
==================================================================

Based on similar historical issues, here is the recommendation:
  - Match Score:           83.45%
  - Relevant Department:   **Mechanical**
  - Estimated Duration:    **Medium**
  - Potential Root Causes: Faulty spark plug or injector

----------------------- FINAL ISSUE SUMMARY ------------------------
- Observed Symptom                : Engine running rough, feels like a misfire
- Error Codes                     : P0301
- Key Sensor Readings/Parameters  : Cylinder 1 EGT was low
- Test Operating Point            : 2000 RPM, 50 Nm
- Engine ID                       : Eng_007
- Test ID                         : Perf_Sweep_3
==================================================================

[INFO] You can export this report with '!export', start a new diagnosis with '!new', or 'quit'.

You: quit
2025-07-02 23:23:45,300 - INFO - __main__ - User requested to quit the application.

Chatbot: Goodbye!

(venv) C:\Projects\Chatbot>